# Sandbox: Using list as default parameter in `__init__`

# The Problem

Currently, our `Directive` class is defined as:

In [20]:
class Directive:
    def __init__(self, name, args=[]):
        self.name = name
        self.args = args
        self.block = []

    def __repr__(self):
        return f"{self.__class__.__name__}(name={self.name!r}, args={self.args!r})"

This causes a problem since `args` takes in a mutable value as the default parameter. Here is the demonstration. To start, let defind two directives:

In [21]:
d1 = Directive("D1")
d2 = Directive("D2")
print(d1)
print(d2)

Directive(name='D1', args=[])
Directive(name='D2', args=[])


At this point, let's alter `d2.args` and see what happens:

In [22]:
d2.args.append("blah")
print(d1)
print(d2)

Directive(name='D1', args=['blah'])
Directive(name='D2', args=['blah'])


As you can see, both `d1` and `d2` shares the same `args`. Worse yet, let's define another directive, `d3`:

In [23]:
d3 = Directive("Zoo")
print(d3)

Directive(name='Zoo', args=['blah'])


# How to Fix It?

In order to fix this problem, we should not assign args to a mutable object such as a list, a dictionary, or a set.

In [31]:
class DirectiveV2:
    def __init__(self, name, args=None):
        self.name = name
        self.args = args or []
        self.block = []

    def __repr__(self):
        return f"{self.__class__.__name__}(name={self.name!r}, args={self.args!r})"

Let us now replay the above sequence:

In [26]:
d1 = DirectiveV2("D1")
d2 = DirectiveV2("D2")
print(d1)
print(d2)

DirectiveV2(name='D1', args=[])
DirectiveV2(name='D2', args=[])


Now, alter `d2.args`:

In [27]:
d2.args.append("blah")
print(d1)
print(d2)

DirectiveV2(name='D1', args=[])
DirectiveV2(name='D2', args=['blah'])


Looks like the problem is fixed. Let see if we can safely create a new directive:

In [28]:
d3 = DirectiveV2("Zoo")
print(d3)

DirectiveV2(name='Zoo', args=[])


So, we fixed the bug. Time to celebrate, but before we go, let's take a look at this interaction:

In [29]:
my_args = [1, 2, 3]
d4 = DirectiveV2("D4", my_args)
print(d4)

DirectiveV2(name='D4', args=[1, 2, 3])


In [30]:
# Now, alter my_args
my_args.append(4)
print(d4)

DirectiveV2(name='D4', args=[1, 2, 3, 4])


It seems that we need to make a copy of the list passed into the `__init__` method:

In [34]:
import copy

class DirectiveV3:
    def __init__(self, name, args=None):
        self.name = name
        self.args = copy.deepcopy(args or [])
        self.block = []

    def __repr__(self):
        return f"{self.__class__.__name__}(name={self.name!r}, args={self.args!r})"

In [35]:
# Test the sequence again
d1 = DirectiveV3("D1")
d2 = DirectiveV3("D2")
print(d1)
print(d2)

DirectiveV3(name='D1', args=[])
DirectiveV3(name='D2', args=[])


In [36]:
# Alter d2
d2.args.append("blah")
print(d1)
print(d2)

DirectiveV3(name='D1', args=[])
DirectiveV3(name='D2', args=['blah'])


In [37]:
# What about the my_args problem
my_args = [1, 2, 3]
d4 = DirectiveV3("D4", my_args)
print(d4)

DirectiveV3(name='D4', args=[1, 2, 3])


In [38]:
my_args.append(4)
print(d4)

DirectiveV3(name='D4', args=[1, 2, 3])
